In [ ]:
# !pip install ortools
from ortools.sat.python import cp_model
from typing import List, Dict, Tuple
import math
import numpy as np

# ----------------------------
# INPUTS (edit these easily)
# ----------------------------
DAYS = 28
SHIFTS = ["M","A","N"]                    # Morning, Afternoon, Night
S = len(SHIFTS)

# Example daily coverage needs (paper case uses M=6, A=6, N=2)
coverage = {"M":6, "A":6, "N":2}         # per day (same each day here; make a list[day] if variable)

# Nurses
nurses = [
    # (name, level1_experienced, is_head)
    ("Alice", True,  True),  # head nurse example
    ("Bee",   True,  False),
    ("Chai",  True,  False),
    ("Dao",   False, False),
    ("Eve",   True,  False),
    ("Fern",  False, False),
    ("Gus",   False, False),
    ("Hana",  True,  False),
    ("Ink",   False, False),
    ("Jay",   True,  False),
    ("Kim",   False, False),
    ("Lek",   True,  False),
    ("Mew",   False, False),
    ("Nok",   True,  False),
    ("Om",    False, False),
    ("Pam",   True,  False),
    ("Q",     False, False),
    ("Ray",   True,  False),
    ("Sue",   False, False),
    ("Tao",   True,  False),
]   # <- Add/remove as needed

N = len(nurses)
name = [n[0] for n in nurses]
is_lvl1 = [n[1] for n in nurses]
is_head = [n[2] for n in nurses]

# Shift codes
M, A, Nshift = 0, 1, 2

# Calendar helpers (0=Mon,...,6=Sun). Adjust start weekday if needed.
def weekday(d): return d % 7

# Preference inputs (0 = neutral, positive = wants this, negative = avoid)
# Example: dict[(nurse_index, day, shift)] = weight
preferences: Dict[Tuple[int,int,int], int] = {
    # Example: Bee prefers Mondays off; reward if not assigned on day 0
    # We'll handle "off" preference via a penalty when assigned; set positive reward for x==0 equivalently.
    # Simpler: negative weights for assignments the nurse dislikes:
    # (n, d, s): -10
}
# Requests for days off (hard? we treat soft in GP): set of (n,d)
requested_dayoff = set([
    # (nurse_index, day)
    # (1, 6),  # Bee requests Sunday off, for example
])

# ----------------------------
# MODEL
# ----------------------------
model = cp_model.CpModel()

# Decision: x[n,d,s] in {0,1} nurse n works shift s on day d
x = {}
for n in range(N):
    for d in range(DAYS):
        for s in range(S):
            x[(n,d,s)] = model.NewBoolVar(f"x[{n},{d},{s}]")

# --- HARD CONSTRAINTS (from paper) ---

# 1) Coverage per shift/day
for d in range(DAYS):
    for s_name, req in coverage.items():
        s = SHIFTS.index(s_name)
        model.Add(sum(x[(n,d,s)] for n in range(N)) == req)

# 2) One shift per day per nurse
for n in range(N):
    for d in range(DAYS):
        model.Add(sum(x[(n,d,s)] for s in range(S)) <= 1)

# 3) Experience mix: at least half of assigned nurses in each shift must be level-1
for d in range(DAYS):
    for s_name, req in coverage.items():
        s = SHIFTS.index(s_name)
        lvl1_count = sum(is_lvl1[n] * x[(n,d,s)] for n in range(N))
        # "At least half": lvl1_count >= ceil(req/2)
        model.Add(lvl1_count >= math.ceil(req/2))

# 4) Monthly workload bounds per nurse: 22..24 shifts
for n in range(N):
    total_shifts = sum(x[(n,d,s)] for d in range(DAYS) for s in range(S))
    model.Add(total_shifts >= 22)
    model.Add(total_shifts <= 24)

# 5) Weekly rest: at least one day OFF each week (7-day block)
for n in range(N):
    for w_start in [0,7,14,21]:
        # "off" means sum of shifts that day = 0 for some d in block
        # Use reified OR across days: introduce bool y_d = "works any shift on day d"
        y_d = [model.NewBoolVar(f"works[{n},{d}]") for d in range(w_start, min(w_start+7, DAYS))]
        for idx, d in enumerate(range(w_start, min(w_start+7, DAYS))):
            model.Add(sum(x[(n,d,s)] for s in range(S)) >= 1).OnlyEnforceIf(y_d[idx])
            model.Add(sum(x[(n,d,s)] for s in range(S)) == 0).OnlyEnforceIf(y_d[idx].Not())
        # Need at least one NOT working day => sum(y_d) <= len(y_d)-1
        model.Add(sum(y_d) <= len(y_d) - 1)

# 6) Night limits per week: ≤2 N per nurse
for n in range(N):
    for w_start in [0,7,14,21]:
        model.Add(sum(x[(n,d,Nshift)] for d in range(w_start, min(w_start+7, DAYS))) <= 2)

# 7) No consecutive nights
for n in range(N):
    for d in range(DAYS-1):
        model.Add(x[(n,d,Nshift)] + x[(n,d+1,Nshift)] <= 1)

# 8) Head-nurse rule: head nurse works Morning Mon–Sat, off Sun
for n in range(N):
    if is_head[n]:
        for d in range(DAYS):
            if weekday(d) in [0,1,2,3,4,5]:  # Mon–Sat
                model.Add(x[(n,d,M)] == 1)
                model.Add(x[(n,d,A)] == 0)
                model.Add(x[(n,d,Nshift)] == 0)
            else:  # Sunday off
                model.Add(sum(x[(n,d,s)] for s in range(S)) == 0)

# (Optional) Soft handling of day-off requests: we'll score violations in the objective
viol_dayoff = {}
for (n,d) in requested_dayoff:
    b = model.NewBoolVar(f"viol_dayoff[{n},{d}]")   # 1 if assigned any shift on requested off day
    model.Add(sum(x[(n,d,s)] for s in range(S)) >= 1).OnlyEnforceIf(b)
    model.Add(sum(x[(n,d,s)] for s in range(S)) == 0).OnlyEnforceIf(b.Not())
    viol_dayoff[(n,d)] = b

# ----------------------------
# GOAL PROGRAMMING LAYER (soft objectives, after feasibility)
# We’ll use a single weighted objective to emulate priority order:
#   Priority 1 (dominates): minimize requested day-off violations
#   Priority 2: maximize preference satisfaction
#   Priority 3: fairness: balance total shifts and night load across nurses
# You can also run “lexicographic” solves in stages if you prefer true preemptive GP.
# ----------------------------

# Preference score: sum of weights for assigned disliked/liked shifts
pref_penalty_terms = []
for (n,d,s), w in preferences.items():
    # If w < 0 (dislike), adding (-w) * x penalizes assignment; if w > 0, we subtract w*x to reward.
    # We'll build a single "penalize bad things" objective: convert positive rewards to negative penalty.
    if w >= 0:
        pref_penalty_terms.append(0)  # handled separately if you want maximize
    else:
        pref_penalty_terms.append((-w) * x[(n,d,s)])

# Workload fairness: minimize deviation from average total shifts
avg_target = np.mean([coverage["M"]+coverage["A"]+coverage["N"] for _ in range(DAYS)]) * 1.0 * 1.0 / N
# That equals (sum required per day * days) / N = ( (6+6+2)*28 ) / N in this example
total_required = (coverage["M"]+coverage["A"]+coverage["N"]) * DAYS
avg_target = total_required / N

dev_pos = []
dev_neg = []
tot = []
for n in range(N):
    t = sum(x[(n,d,s)] for d in range(DAYS) for s in range(S))
    tot.append(t)
    dp = model.NewIntVar(0, DAYS, f"dev_pos[{n}]")
    dn = model.NewIntVar(0, DAYS, f"dev_neg[{n}]")
    # t - avg_target = dp - dn
    # Need integer constants: use rounded avg_target
    tgt = int(round(avg_target))
    model.Add(t - tgt == dp - dn)
    dev_pos.append(dp)
    dev_neg.append(dn)

# Night workload fairness: spread night duties
night_dev = []
for n in range(N):
    tn = sum(x[(n,d,Nshift)] for d in range(DAYS))
    # target nights ~ total night demand / N
    night_total = coverage["N"] * DAYS
    night_tgt = int(round(night_total / N)) if N > 0 else 0
    ndp = model.NewIntVar(0, DAYS, f"night_dev_pos[{n}]")
    ndn = model.NewIntVar(0, DAYS, f"night_dev_neg[{n}]")
    model.Add(tn - night_tgt == ndp - ndn)
    night_dev.append((ndp, ndn))

# Build weighted penalty objective (Preemptive GP emulation)
# Large weights enforce priority order: W1 >> W2 >> W3
W1 = 10_000  # day-off request violations
W2 = 1_000   # preferences (penalize dislikes)
W3 = 100     # workload fairness (total shifts)
W4 = 50      # night fairness

penalty = 0

# Priority 1: requested day-off violations
if viol_dayoff:
    penalty += W1 * sum(viol_dayoff.values())

# Priority 2: preferences (penalize disliked assigned shifts)
if pref_penalty_terms:
    penalty += W2 * sum(pref_penalty_terms)

# Priority 3: workload fairness (L1 deviation)
penalty += W3 * sum(dev_pos) + W3 * sum(dev_neg)

# Priority 4: night fairness
penalty += W4 * sum(ndp for (ndp, ndn) in night_dev) + W4 * sum(ndn for (ndp, ndn) in night_dev)

# Finalize objective (minimize penalty)
model.Minimize(penalty)

# ----------------------------
# SOLVE
# ----------------------------
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 60.0
solver.parameters.num_search_workers = 8
status = solver.Solve(model)

print("Status:", solver.StatusName(status))
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    # Pretty print schedule
    by_day = {d: {s: [] for s in range(S)} for d in range(DAYS)}
    for d in range(DAYS):
        for s in range(S):
            for n in range(N):
                if solver.Value(x[(n,d,s)]) == 1:
                    by_day[d][s].append(name[n])

    for d in range(DAYS):
        wk = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"][d%7]
        print(f"\nDay {d+1:02d} ({wk})")
        for s in range(S):
            print(f"  {SHIFTS[s]}: {by_day[d][s]}")

    # Objective diagnostics
    tot_shifts = [solver.Value(sum(x[(n,d,s)] for d in range(DAYS) for s in range(S))) for n in range(N)]
    nights = [solver.Value(sum(x[(n,d,Nshift)] for d in range(DAYS))) for n in range(N)]
    print("\nTotal required:", total_required, "Avg target per nurse:", int(round(avg_target)))
    print("Per-nurse total shifts:", dict(zip(name, tot_shifts)))
    print("Per-nurse nights:", dict(zip(name, nights)))
else:
    print("No feasible solution under current constraints/weights.")


Status: OPTIMAL

Day 01 (Mon)
  M: ['Alice', 'Fern', 'Hana', 'Nok', 'Q', 'Sue']
  A: ['Bee', 'Eve', 'Gus', 'Ink', 'Lek', 'Tao']
  N: ['Jay', 'Mew']

Day 02 (Tue)
  M: ['Alice', 'Dao', 'Fern', 'Lek', 'Om', 'Ray']
  A: ['Bee', 'Gus', 'Jay', 'Kim', 'Pam', 'Q']
  N: ['Hana', 'Ink']

Day 03 (Wed)
  M: ['Alice', 'Chai', 'Dao', 'Ink', 'Kim', 'Tao']
  A: ['Bee', 'Fern', 'Gus', 'Hana', 'Nok', 'Q']
  N: ['Om', 'Ray']

Day 04 (Thu)
  M: ['Alice', 'Fern', 'Gus', 'Ink', 'Pam', 'Tao']
  A: ['Bee', 'Chai', 'Eve', 'Kim', 'Nok', 'Q']
  N: ['Lek', 'Sue']

Day 05 (Fri)
  M: ['Alice', 'Chai', 'Dao', 'Om', 'Ray', 'Sue']
  A: ['Fern', 'Gus', 'Ink', 'Nok', 'Pam', 'Tao']
  N: ['Eve', 'Kim']

Day 06 (Sat)
  M: ['Alice', 'Chai', 'Dao', 'Fern', 'Om', 'Pam']
  A: ['Eve', 'Kim', 'Lek', 'Mew', 'Nok', 'Tao']
  N: ['Jay', 'Sue']

Day 07 (Sun)
  M: ['Chai', 'Hana', 'Mew', 'Om', 'Ray', 'Sue']
  A: ['Bee', 'Dao', 'Eve', 'Jay', 'Q', 'Tao']
  N: ['Gus', 'Nok']

Day 08 (Mon)
  M: ['Alice', 'Fern', 'Jay', 'Nok', 'Om', 'Tao'